In [1]:
import pandas as pd
import numpy as np
import copy ,time
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = 'data/'
df_train = pd.read_csv(data_path+'train_data.csv')
df_test = pd.read_csv(data_path+'test_features.csv')

In [2]:
train_Y = df_train['poi']

In [3]:
df_train['poi']
ids = df_test['name']

df_train = df_train.drop(['name','poi'],axis=1)

In [4]:
df_test = df_test.drop(['name'],axis=1)
df = pd.concat([df_train,df_test])


In [5]:
df

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,4.0,NaN,1617011.0,174839.0,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,NaN,1920000.0,22122.0,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,14.0,NaN,NaN,1573324.0,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,NaN,NaN,602671.0,907502.0,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,25.0,NaN,375304.0,486.0,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,1000000.0,NaN,NaN,NaN,philippe.bibi@enron.com,1465734.0,38559.0,40.0,23.0,8.0,NaN,369721.0,425688.0,378082.0,NaN,213625.0,1336.0,1607.0,2047593.0,1843816.0
29,1500000.0,NaN,NaN,NaN,john.sherriff@enron.com,1835558.0,NaN,92.0,28.0,23.0,NaN,554422.0,1852186.0,1293424.0,NaN,428780.0,2103.0,3187.0,4335388.0,3128982.0
30,NaN,504610.0,NaN,NaN,dana.gibbs@enron.com,2218275.0,NaN,12.0,0.0,0.0,NaN,461912.0,NaN,NaN,NaN,NaN,23.0,169.0,966522.0,2218275.0
31,200000.0,204075.0,NaN,NaN,tod.lindholm@enron.com,2549361.0,57727.0,NaN,NaN,NaN,NaN,175000.0,2630.0,514847.0,NaN,236457.0,NaN,NaN,875889.0,3064208.0


In [6]:
df.columns

Index(['bonus', 'deferral_payments', 'deferred_income', 'director_fees',
       'email_address', 'exercised_stock_options', 'expenses', 'from_messages',
       'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
       'long_term_incentive', 'other', 'restricted_stock',
       'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
       'to_messages', 'total_payments', 'total_stock_value'],
      dtype='object')

In [7]:
def na_check(df_data):
    data_na = (df_data.isnull().sum()/len(df_data))*100
    data_na = data_na.drop(data_na[data_na==0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio':data_na})
    display(missing_data)
na_check(df)

,Missing Ratio
loan_advances,97.260274
director_fees,88.356164
restricted_stock_deferred,87.671233
deferral_payments,73.287671
deferred_income,66.438356
long_term_incentive,54.794521
bonus,43.835616
from_messages,41.095890
from_poi_to_this_person,41.095890
from_this_person_to_poi,41.095890


In [8]:
pd.options.display.max_rows = None

In [9]:
df['loan_advances']

0             NaN
1             NaN
2             NaN
3             NaN
4             NaN
5             NaN
6             NaN
7             NaN
8      81525000.0
9             NaN
10            NaN
11            NaN
12            NaN
13            NaN
14            NaN
15            NaN
16            NaN
17            NaN
18            NaN
19            NaN
20            NaN
21            NaN
22            NaN
23            NaN
24            NaN
25            NaN
26            NaN
27            NaN
28            NaN
29            NaN
30            NaN
31            NaN
32            NaN
33            NaN
34            NaN
35            NaN
36            NaN
37            NaN
38            NaN
39            NaN
40            NaN
41            NaN
42            NaN
43            NaN
44            NaN
45            NaN
46            NaN
47            NaN
48            NaN
49            NaN
50            NaN
51            NaN
52            NaN
53            NaN
54            NaN
55        

In [10]:
df = df.fillna(0)

In [11]:
df

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,loan_advances,long_term_incentive,other,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,1750000.0,0.0,-3504386.0,0.0,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,4.0,0.0,1617011.0,174839.0,2748364.0,0.0,420636.0,864.0,905.0,505050.0,22542539.0
1,5600000.0,0.0,0.0,0.0,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,30.0,0.0,1920000.0,22122.0,6843672.0,0.0,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,200000.0,0.0,-4167.0,0.0,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,14.0,0.0,0.0,1573324.0,869220.0,0.0,211844.0,91.0,225.0,2003885.0,2493616.0
3,800000.0,0.0,0.0,0.0,michael.kopper@enron.com,0.0,118134.0,0.0,0.0,0.0,0.0,602671.0,907502.0,985032.0,0.0,224305.0,0.0,0.0,2652612.0,985032.0
4,1250000.0,0.0,-262500.0,0.0,christopher.calger@enron.com,0.0,35818.0,144.0,199.0,25.0,0.0,375304.0,486.0,126027.0,0.0,240189.0,2188.0,2598.0,1639297.0,126027.0
5,0.0,10259.0,0.0,0.0,joe.hirko@enron.com,30766064.0,77978.0,0.0,0.0,0.0,0.0,0.0,2856.0,0.0,0.0,0.0,0.0,0.0,91093.0,30766064.0
6,0.0,0.0,0.0,0.0,scott.yeager@enron.com,8308552.0,53947.0,0.0,0.0,0.0,0.0,0.0,147950.0,3576206.0,0.0,158403.0,0.0,0.0,360300.0,11884758.0
7,1200000.0,27610.0,-144062.0,0.0,wes.colwell@enron.com,0.0,16514.0,40.0,240.0,11.0,0.0,0.0,101740.0,698242.0,0.0,288542.0,1132.0,1758.0,1490344.0,698242.0
8,7000000.0,202911.0,-300000.0,0.0,kenneth.lay@enron.com,34348384.0,99832.0,36.0,123.0,16.0,81525000.0,3600000.0,10359729.0,14761694.0,0.0,1072321.0,2411.0,4273.0,103559793.0,49110078.0
9,600000.0,0.0,0.0,0.0,ben.glisan@enron.com,384728.0,125978.0,16.0,52.0,6.0,0.0,71023.0,200308.0,393818.0,0.0,274975.0,874.0,873.0,1272284.0,778546.0


In [12]:
df.drop(labels = ["email_address"], axis = 1, inplace = True)

In [14]:
df.drop(labels = ["to_messages"], axis = 1, inplace = True)

In [15]:
df.columns

Index(['bonus', 'deferral_payments', 'deferred_income', 'director_fees',
       'exercised_stock_options', 'expenses', 'loan_advances',
       'long_term_incentive', 'other', 'restricted_stock',
       'restricted_stock_deferred', 'salary', 'total_payments',
       'total_stock_value'],
      dtype='object')

In [16]:
df = MinMaxScaler().fit_transform(df)

In [17]:
train_Y.shape[0]

113

In [18]:
train_num = train_Y.shape[0]
train_X = df[:train_num]
test_X = df[train_num:]

In [19]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100,min_samples_split=2,min_samples_leaf=1,
                           max_features='sqrt',max_depth=6,bootstrap=True)

In [20]:
rf.fit(train_X,train_Y)
rf_pred = rf.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name':ids,'poi':rf_pred})
sub.to_csv('submit_rf.csv',index=False)

In [30]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(tol=0.001,penalty='l2',fit_intercept=True,C=1.0)

In [33]:
lr.fit(train_X,train_Y)
lr_pred = lr.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name':ids,'poi':lr_pred})
sub.to_csv('submit_lr.csv',index=False)

In [36]:
from sklearn.ensemble import GradientBoostingClassifier

gdbt = GradientBoostingClassifier(tol=100,subsample=0.75,n_estimators=250,max_features=10,
                                 max_depth=6,learning_rate=0.03)

In [37]:
gdbt.fit(train_X,train_Y)
gdbt_pred = gdbt.predict_proba(test_X)[:,1]
sub = pd.DataFrame({'name':ids,'poi':gdbt_pred})
sub.to_csv('submit_gdbt.csv',index=False)